In [ ]:
import math
import random
import functools
from matplotlib import pyplot as plt

# Algoritmo Genético para Otimização de uma Função Univariada

O Código a seguir implementa um algoritmo genético para otimizar uma função univariada. O objetivo é mostrar, usando um problema simples, o papel das funções do algoritmo genético. No caso do exemplo, o algoritmo computa

$$\max f(x) \, , \, f(x) = \sin \left( \frac{\pi x}{256} \right ), x \in [0, 1, \dots 255]$$

In [18]:
#Função de aptitude: quanto maior o valor melhor é a solução. Obviamente, ao maximizar uma função,
#a própria função pode ser usada como função de aptitude.
def fitness(x):
    return math.sin((math.pi * x) / 256)
 
#Gera uma população aleatória com N elementos no intervalo [0,255]
def gen_pop(N):
    pop = []
    for i in range(N):
        pop.append(random.randint(0,255))
    return pop

#Retorna uma função que retorna True com probabilidade prob/100.
def mut_prob_constante(prob):
    def uniform():
        return True if random.randint(0,100) < prob else False
    return uniform

#Função de reprodução com um único corte (crossover de um ponto)
def reproduzir(x,y):
    gx = format(x, '08b')
    gy = format(y, '08b')
    corte = random.randint(0, len(gx)-1)
    novo = gx[0:corte] + gy[corte:len(gx)]
    return int(novo, 2)

#Função auxiliar para computar a FDA (Função de distribuição acumulada)
def acumular(v):
    acum = 0
    r = []
    for i in v:
        r.append(i + acum)
        acum = r[-1]
    return r

#Função de seleção de indivíduo que escolhe com maior probabilidade os indivíduos mais aptos
def selecionar_individuo(pop, f):
    fits = list(map(f, pop))
    soma = functools.reduce(lambda x,y: x + y, fits)
    norms = list(map(lambda x,y: x / y, pop, [soma] * len(pop)))
    acum = list(acumular(norms))
    r = random.random()
    for i in range(len(acum)):
        if r < acum[i]:
            break
    return pop[i]

#Realiza uma mutação: inverte um bit aleatório da solução
def mutar(ind):
    gi = format(ind, '08b')
    r = random.randrange(len(gi))
    return ind ^ (1 << r)

def argmax(v):
    m = 0
    for i in range(len(v)):
        if v[i] > v[m]:
            m = i
    return m

#Algoritmo genético clássico, como em Russell e Norvig(2010)
def genetico(pop_inicial, f, mut_prob_f, iters=100, graph=True):
    pop = pop_inicial
    max_fits = []
    mean_fits = []
    best_solution = (pop[argmax(list(map(f,pop)))])
    
    for i in range(iters):
        nova_pop = []
        for p in range(len(pop)):
            x = selecionar_individuo(pop, f)
            y = selecionar_individuo(pop, f)
            filho = reproduzir(x,y)
            if mut_prob_f():
                filho = mutar(filho)
            nova_pop.append(filho)
        pop = nova_pop
        fits = list(map(f, pop))
        max_fits.append(max(fits))
        mean_fits.append(sum(fits) / len(fits))
        
        if f(max_fits[-1]) > f(best_solution):
            best_solution = max_fits[-1]
        
    if graph:
        %matplotlib tk
        plt.figure()
        plt.plot(range(iters), mean_fits, label='Fitness medio')
        plt.plot(range(iters), max_fits, label='Fitness maximo')
        plt.legend()
        plt.ylabel('Fitness')
        plt.xlabel('Geracoes')
        plt.show()   
        
    return pop, best_solution


pop_inicial = [10, 30, 90, 170, 200, 100, 97, 2, 45, 110] #gen_pop(10)
print(pop_inicial)
print(list(map(fitness, pop_inicial)), sum(map(fitness,pop_inicial)) / len(pop_inicial))
mutacao_prob_f = mut_prob_constante(5)
res, melhor = genetico(pop_inicial, fitness, mutacao_prob_f, iters=100)
print(res, melhor, fitness(melhor))
print(list(map(fitness, res)), sum(map(fitness,res)) / len(res))

print("Melhor solução encontrada: %d, f(%d) = %.3f" % (melhor, melhor, fitness(melhor)))

#print(mutar(10))
#p = gen_pop(10)    
#selecionar_individuo(p,fitness)

[10, 30, 90, 170, 200, 100, 97, 2, 45, 110]
([0.1224106751992162, 0.3598950365349881, 0.8932243011955153, 0.8700869911087115, 0.6343932841636455, 0.9415440651830208, 0.9285060804732155, 0.024541228522912288, 0.524589682678469, 0.9757021300385286], 0.6274893475098222)
([153, 153, 153, 153, 153, 153, 153, 153, 153, 153], 110, 0.9757021300385286)
([0.9533060403541939, 0.9533060403541939, 0.9533060403541939, 0.9533060403541939, 0.9533060403541939, 0.9533060403541939, 0.9533060403541939, 0.9533060403541939, 0.9533060403541939, 0.9533060403541939], 0.953306040354194)
Melhor solução encontrada: 110, f(110) = 0.976
